#### CSV to LSTM data

File to transform csv data into json files formatted for the training of a lstm.

Within the file the column names needed for the magnet temperature are used.

In [ ]:
import numpy as np
from joblib import load, dump
import json

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import sklearn as sk

import glob
import os

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA

from joblib import dump, load
 
import tensorflow as tf
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.activations import relu, tanh, linear
from keras.layers import Dropout
from sklearn.neural_network import MLPRegressor

from keras.callbacks import EarlyStopping, CSVLogger, TerminateOnNaN
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer

import json

In [ ]:
from readIn import readIn

In [ ]:
trainDataPath = "X:\\RotorTempKI\\train"
valDataPath = "X:\\RotorTempKI\\validation"

# Pipeline
pathname = "X:\\RotorTempKI\\pipeline.p"

filenames_train = [["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s5_w50_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s5_w50_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s6_w50_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s6_w50_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s7_w50_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s7_w50_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s5_w60_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s5_w60_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s6_w60_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s6_w60_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s7_w60_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s7_w60_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s5_w70_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s5_w70_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s6_w70_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s6_w70_y.json"],
                   ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s7_w70_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\train\\train_s7_w70_y.json"]]
filenames_val = [["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s5_w50_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s5_w50_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s6_w50_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s6_w50_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s7_w50_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s7_w50_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s5_w60_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s5_w60_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s6_w60_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s6_w60_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s7_w60_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s7_w60_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s5_w70_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s5_w70_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s6_w70_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s6_w70_y.json"],
                ["C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s7_w70_x.json", "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\validation\\val_s7_w70_y.json"]]

In [ ]:
# generateDataSource() -> 3D array von den Daten
### window  represents time period by each entry in the buffer
### Shift represents the jump from value to next one in the buffer
### sampling rate

def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1, sample_rate=1):
    #subsequence_len= (window -1) *shift + 1
    subsequence_len= (window) *shift
    Signal_Length = signal_input.shape[0]
    num_samples = 1 + int((Signal_Length - subsequence_len) / sample_rate)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = np.asarray([signal_input[i*sample_rate + j * shift] for j in range(0,window)])
        y[i] = signal_output[i*sample_rate + (window-1) * shift :i*sample_rate+ (window-1) * shift + output_length]
    return x, y

In [ ]:
# Using Mustafas function
def SaveJsonWindows(filenames, shift, window, x_transform, y_train, x_validation = None, y_validation = None):
    i = 0
    if x_validation != None:
        x_in = np.concatenate((x_transform, x_validation))
    else: 
        x_in = x_transform
    if y_validation != None:
        y_in = np.concatenate((y_train, y_validation))
    else: 
        y_in = y_train

    for w in window:
        for s in shift:
            x_cv, y_cv = generateDataSource(signal_input = x_in,
                                            signal_output=y_in, 
                                            window = w, shift=s, sample_rate=1)
            out_x = open(filenames[i][0], mode="x")
            json.dump(x_cv.tolist(), out_x)
            out_x.close()
            out_y = open(filenames[i][1], mode="x")
            json.dump(y_cv.tolist(), out_y)
            out_y.close()
            i += 1

In [ ]:
def OpenJsontoArr(path):
    file = open(path)           #"X:\\KI Praktikum\\validate_Data\\3darray_x_cv.json"
    x_3d = json.load(file)
    file.close()
    x_3d = np.asarray(x_3d)
    return x_3d

In [ ]:
df_train = readIn(trainDataPath)
df_val = readIn(valDataPath)

In [ ]:
print(df_train.head(5))
df_val = df_val.drop(columns=["start", "end"])
print(df_val.head(5))

In [ ]:
y_train = df_train["magnet_temperature"]
parameters = ["T_KLEMMUNG", "T_LAGER", "DRZ5", "T_MOTOR", "I_FELD", "V_QUER"]
x_train = df_train[parameters].to_numpy()

y_val = df_val["magnet_temperature"]
x_val = df_val[parameters].to_numpy()

In [ ]:
seq_steps = [("standardize_values", StandardScaler()), ("pca", PCA(6)), ("standard_values", StandardScaler())]
pipeline = Pipeline(steps=seq_steps)
dump(pipeline, pathname)

In [ ]:
newpipeline = load(pathname)
x_transform = newpipeline.fit_transform(x_train)
x_test = newpipeline.transform(x_val)
newpipeline.steps[1][1].explained_variance_ratio_

In [ ]:
SaveJsonWindows(filenames=filenames_train, shift=[5, 6, 7], window=[50, 60, 70], x_transform=x_transform, y_train=y_train)
SaveJsonWindows(filenames=filenames_val, shift=[5, 6, 7], window=[50, 60, 70], x_transform=x_test, y_train=y_val)